In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

## RunnablePassthrough
* It does not do anything to the input data.
* Let's see it in a very simple example: a chain with just RunnablePassthrough() will output the original input without any modification.

In [3]:
from langchain_core.runnables import RunnablePassthrough
chain=RunnablePassthrough()

In [4]:
chain.invoke("prince katiyar")

'prince katiyar'

## RunnableLambda
* To use a custom function inside a LCEL chain we need to wrap it up with RunnableLambda.
* Let's define a very simple function to create Russian lastnames:

In [6]:
def russian_lastname(name: str) -> str:
    return f"{name}  ovich"

In [7]:
from langchain_core.runnables import RunnableLambda

chain = RunnablePassthrough() | RunnableLambda(russian_lastname)

In [8]:
chain.invoke("prince katiyar")

'prince katiyar  ovich'

## RunnableParallel
* We will use RunnableParallel() for running tasks in parallel.
* This is probably the most important and most useful Runnable from LangChain.
* In the following chain, RunnableParallel is going to run these two tasks in parallel:
    * operation_a will use RunnablePassthrough.
    * operation_b will use RunnableLambda with the russian_lastname function.

In [9]:
from langchain_core.runnables import RunnableParallel

In [10]:
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "operation_b": RunnableLambda(russian_lastname)
    }
)

In [11]:
chain.invoke("prince")

{'operation_a': 'prince', 'operation_b': 'prince  ovich'}

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [13]:
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

In [14]:
def russian_lastname_from_dictionary(person):
    return person["name"] + "ovich"

In [15]:
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "soccer_player": RunnableLambda(russian_lastname_from_dictionary),
        "operation_c": RunnablePassthrough(),
    }

    
) | prompt | model | output_parser

In [16]:
chain.invoke({
    "name1": "Jordam",
     "name": "Abram"})

'A curious fact about Roman Abramovich is that he is known for his massive collection of luxury yachts, with some estimates suggesting he owns up to seven yachts at any given time. These yachts are often seen cruising in the Mediterranean and Caribbean seas, showcasing his extravagant lifestyle.'

In [17]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_openai import ChatOpenAI, OpenAIEmbeddings


In [ ]:
vectorstore = FAISS.from_texts(
    ["learning logic  focuses on providing content on Data Science, AI, ML, DL, CV, NLP, Python programming, etc. in English."], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""


prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(model="gpt-3.5-turbo")

retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)
retrieval_chain.invoke("What is is learning logic ")

'Learning logic is a platform that provides content on Data Science, AI, ML, DL, CV, NLP, Python programming, etc. in English.'

In [19]:
print("The End")

The End
